### NOTES
- Must find optimization opportunities
    - The rate of the current routine is 0.6 conformers / 1 second on laptop CPU
    - The comparable moleculekit rate is 6 conformers / 1 second on laptop CPU
    - An ideal rate would be ~ 10 conformers / 1 second on laptop CPU
    - Optimization opportunities:
        - Parallelization (not directly supported by moleculekit)
        - GPU via moleculekit CUDA bindings for voxel descriptors
            - Might actually run **slower** considering overhead
        - Removing safety check wrappers (_asSmallMol, _Conformer) and copy statements
        - Direct RDKit method invocation
- Does generateConformers(1) generate the lowest energy conformer?
- All conformers are (erroneously) assigned the same response value
  - This implies that all conformers bind with the same affinity, which is not true
  - The conformation of the bound ligand is constrained, and may not be the lowest energy conformation
  - The bound conformation could be estimated by docking, but its is not feasible from a time perspective
      - This would defeat the purpose of ML modelling anyway

In [ ]:
import numpy as np
from math import sqrt
from copy import deepcopy
from itertools import repeat

from moleculekit.util import boundingBox, orientOnAxes
from moleculekit.smallmol.smallmol import SmallMol
from moleculekit.tools.voxeldescriptors import getVoxelDescriptors

from rdkit.Chem import Conformer

In [ ]:
def _rdkit_TransformMol(mol, tform, confId):
    newConf = Conformer()
    newConf.SetId(0)
    refConf = mol.GetConformer(confId)
    for i in range(refConf.GetNumAtoms()):
        pos = list(refConf.GetAtomPosition(i))
        newPos = np.dot(tform, np.array(pos))
        newConf.SetAtomPosition(i, list(newPos)[:3])
    return newConf

def _orientOnAxes(smol):
    smol = smol.copy()
    nconfs = smol.numFrames
    mol = deepcopy(smol._mol)
    for idx in range(nconfs):
        coords = smol._coords[:,:,idx]
        covariance = np.cov(coords.T)
        eigenvalues, eigenvectors = np.linalg.eigh(covariance)
        if np.linalg.det(eigenvectors) < 0: 
            eigenvectors = -eigenvectors
        conf = _rdkit_TransformMol(mol, eigenvectors.T, idx)
        conf.SetId(idx)
        mol.RemoveConformer(idx)
        mol.AddConformer(conf, assignId=False)
    smol._mol = mol
    return smol

In [ ]:
def _isSmallMol(x):
    return isinstance(x, SmallMol) or type(x).__name__ is 'SmallMol'

def _asSmallMol(x):
    """ 
    Creates a moleculekit.smallmol.smallmol.SmallMol(x) instance if `x` is not already a SmallMol instance.
    Parameters
    ----------
    x : object
        One of the following (i) Rdkit molecule or (ii) Location of molecule file (“.pdb”/”.mol2”) or (iii) a smile string or iv) another SmallMol object or v) moleculekit.molecule.Molecule object.
    Returns
    -------
    moleculekit.smallmol.smallmol.SmallMol
        A moleculekit SmallMol instance created from x and holding at least one conformer, which may be empty.
    """
    smol = x.copy() if _isSmallMol(x) else SmallMol(x).copy()
    if smol.numFrames < 1: smol.generateConformers(num_confs=1) 
    return smol

def _Conformer(x, idx=0):
    """ 
    Extract a single frame from a SmallMol instance. A frame will be created if no frames exist in `x`.
    Parameters
    ----------
    x : object
        One of the following (i) Rdkit molecule or (ii) Location of molecule file (“.pdb”/”.mol2”) or (iii) a smile string or iv) another SmallMol object or v) moleculekit.molecule.Molecule object.
    Returns
    -------
    moleculekit.smallmol.smallmol.SmallMol
        A moleculekit SmallMol instance created from x and holding only a single frame.
    """
    smol = _asSmallMol(x)
    mol = smol.toMolecule(ids=[idx])
    smol = SmallMol(mol)
    return smol

def _SmallMol(x):
    # TODO:: cleanup conformers with RDKit
    """ 
    Create a SmallMol instance from `x` and perfom preprocessing steps.
    Parameters
    ----------
    x : object
        One of the following (i) Rdkit molecule or (ii) Location of molecule file (“.pdb”/”.mol2”) or (iii) a smile string or iv) another SmallMol object or v) moleculekit.molecule.Molecule object.
    Returns
    -------
    moleculekit.smallmol.smallmol.SmallMol
        A moleculekit SmallMol instance created from x and holding at least one conformer.
    """
    smol = _asSmallMol(x)
    smol = _orientOnAxes(smol)
    return smol

In [ ]:
def _max_coords(mol):
    return np.squeeze(np.max(mol.coords, axis=0))

def _min_coords(mol):
    return np.squeeze(np.min(mol.coords, axis=0))

def _min_max_coords(mol):
    minc = _min_coords(mol)
    maxc = _max_coords(mol)
    return minc,maxc

def _boundingBoxSize(minc, maxc):
    eucd = np.linalg.norm(minc-maxc)
    edge = eucd/sqrt(3)
    x,y,z = np.repeat(edge,3)
    return x,y,z

def _boundingBoxCenter(minc, maxc):
    x,y,z = (minc+maxc)/2
    return x,y,z
    
def min_common_box(smols):
    gmin,gmax = np.zeros((2,3))
    for smol in smols:
        smol = _asSmallMol(smol)
        nconfs = smol.numFrames
        for i in range(nconfs): 
            mol = smol.toMolecule(ids=[i])
            lmin,lmax = boundingBox(mol)
            dmin = lmin < gmin
            dmax = lmax > gmax
            if np.any(dmin):
                for i in np.where(dmin):
                    gmin[i] = lmin[i]
            if np.any(dmax):
                for i in np.where(dmax):
                    gmax[i] = lmax[i]
    bbox = (gmin,gmax)
    boxsize = _boundingBoxSize(*bbox)
    center = _boundingBoxCenter(*bbox)
    return bbox,boxsize,center

In [ ]:
def _moleculekit_getPropertiesRDKit(smol, factory, mapping):
    """ 
    A refactored version of `moleculekit.tooks.voxeldescriptors._getPropertiesRDkit` that allows the user to pass their own RDKit feature factory and schema for mapping atom features. Occupancy embedding has been removed.
    Notes
    -----
    Key differences compared to moleculekit's `_getPropertiesRDKit`: 1) No support for Molecule objects 2) Atom occupancies are not computed 3) User-defined feature factory and atom_mappings without default arguments 4) Dynamically computed array dimensions
    This implementation yields 5 channels using the default factory and atom_mappings found in moleculekit's implementation. Moleculekit's original `_getPropertiesRDKit` yields 7 channels with channel 6 being blank (excluding the occupancy embedding).
    I'm pretty sure the moleculekit source code has the indexing wrong. They have 7 computed features with a max channel index of 5 (redundant mapping to index 0), plus 1 occupancy channel. I'd expect the returned array to therefor contain 6 channels and have shape (<n_atoms>, 7), but the returned array actually contains 7 channels and has shape (<n_atoms>, 8).
    Examples
    -----
    >>> import numpy as np
    >>> from moleculekit.tools.voxeldescriptors import _getPropertiesRDkit
    >>> from moleculekit.smallmol.util import factory
    >>> smol = SmallMol('CCO')
    >>> x = _moleculekit_getPropertiesRDKit(smol)
    >>> y = _getPropertiesRDkit(smol)
    >>> np.testing.assert_array_equal(x, y)
    """
    natoms = smol.numAtoms
    nchannels = max(mapping.values()) + 1 # +1 for 0 indexing
    feats = factory.GetFeaturesForMol(smol._mol)
    properties = np.zeros((natoms, nchannels), dtype=bool)
    for feat in feats:
        fam = feat.GetFamily()
        if fam in mapping.keys():
            properties[feat.GetAtomIds(), mapping[fam]] = 20
    return properties

def _default_embedding(smol):
    from moleculekit.smallmol.util import factory
    mapping = {"Hydrophobe": 0, "LumpedHydrophobe": 0, "Aromatic": 1, "Acceptor": 2, "Donor": 3, "PosIonizable": 4, "NegIonizable": 5}
    return _moleculekit_getPropertiesRDKit(smol, factory, mapping)

def _occupancy_embedding(smol):
    natoms = smol.numAtoms
    properties = np.zeros((natoms, 1), dtype=bool)
    properties[:, 0] = smol.get('element') != 'H'
    return properties

def _makeConformerChannels(smol, embeddings=[_default_embedding, _occupancy_embedding]):
    """
    Generate an argument to `userchannels` for a single conformer by applying the functions in `fxns` to `smol`.
    Parameters
    ----------
    smol: moleculekit.smallmol.smallmol.SmallMol
        A `SmallMol` instance containing a single frame (i.e. `smol.numFrames == 1`). If more than one frame is found, only the frame at the index pointed to by the `frame` attribute will be used. All other frames will be ignored without warning.
    Examples
    --------
    >>> from moleculekit.tools.voxeldescriptors import getChannels
    >>> smol = SmallMol('CCO')
    >>> smol.generateConformers(1)
    >>> _makeConformerChannels(smol)
    """
    channels = []
    for fxn in embeddings:
        props = fxn(smol)
        if props.dtype == bool:
            sigmas = _getChannelRadii(smol.get('element'))
            props = sigmas[:, np.newaxis] * props.astype(float)
            channels.append(props)
    return np.hstack((channels))

def makeChannels(smol, embeddings=[_default_embedding, _occupancy_embedding], unique=True):
    """
    Generate an argument to `userchannels` for all conformers in `smol` by applying the functions in `fxns` to each frame in `smol`.
    Parameters
    ----------
    smol: moleculekit.smallmol.smallmol.SmallMol
        A `SmallMol` instance assumed to contain at least one frame. A single frame will be created if none are found.
    Returns
    -------
    numpy.ndarray
        An array of shape (<n_conformers>, <n_atoms>, <n_channels>) representing the computed channels for each conformer in `smol`.
    Examples
    --------
    >>> smol = SmallMol('C([C@@H]1[C@H]([C@@H]([C@H](C(O1)O)O)O)O)O')
    >>> smol.generateConformers(10)
    >>> userchannels = makeChannels(smol, unique=False)
    >>> print('N conformers: %s' % smol.numFrames)
    >>> print('N embeddings: %s' % userchannels.shape)
    >>> userchannels = makeChannels(smol)
    >>> print('N conformers: %s' % smol.numFrames)
    >>> print('N unique embeddings: %s' % userchannels.shape)
    """
    def contains(x, y):
        contains = False
        for a in x:
            if np.array_equal(a, y):
                contains = True
                break
        return contains
    smol = _asSmallMol(smol)
    nconfs = smol.numFrames
    conf_channels = []
    for idx in range(nconfs):
        conf = _Conformer(smol, idx)
        channels = _makeConformerChannels(conf)
        doappend = True
        if unique:
            doappend = not contains(conf_channels, channels)
        if doappend:
            conf_channels.append(channels)
    return np.array(conf_channels)

In [ ]:
def _generateConformers(smol, num_confs=1, optimizemode='mmff', align=True, append=True):
    # TODO:: This is fucky and slow with the copy()
    smol = smol.copy()
    if not append and num_confs > smol.numFrames:
        num_confs = num_confs
    smol.generateConformers(num_confs, optimizemode, align, append)
    return smol
        
def _embed(smol, *args, **kwargs):
    # TODO:: I dont know if *args will work as expected, because it is positionally displaced by the `smol` parameter.
    """Embed a single conformer derived from a SmallMolecule instance 
    smol: moleculekit.smallmol.smallmol.SmallMol
        A `SmallMol` instance containing a single frame (i.e. `smol.numFrames == 1`), or an object coercible to SmallMol. If more than one frame is found, only the frame at the index pointed to by the `frame` attribute will be used. All other frames are ignored without warning.
    """
    vox, centers, N = getVoxelDescriptors(smol, *args, **kwargs)
    grid = vox.transpose().reshape([vox.shape[1], N[0], N[1], N[2]])
    return np.array(grid)

def embed_grid(smol, cargs={}, vargs={}):
    """
    Embed each conformer of a SmallMolecule instance in a discrete 3D grid. Each conformer is embedded in its own grid. Each grid is a multidimensional array of the same dimensions created using the same descriptors for all conformers.
    Parameters
    ----------
    smol : object
        The SmallMol object to be embedded, or any object coerceible to a SmallMol. Coercible objects are: (i) RDKit molecule or (ii) Location of molecule file (“.pdb”/”.mol2”) or (iii) a SMILES string or iv) a SmallMol object or v) moleculekit.molecule.Molecule object
    cargs: dict
        Named arguments passed directly to `SmallMol(smol).generateConformers(**cargs)`. Default key/value pairs are `{num_confs:1, optimizemode:'mmff', align:True, append:True}`.
    vargs: dict
        Named arguments passed directly to `moleculekit.tools.voxeldescriptors.getVoxelDescriptors(**vargs)`.
    Returns
    -------
    numpy.ndarray
        A multidimensional array representing the voxelized embedding of each conformer in `smol`. Dimensions are: (<n_conformers>, <n_channels>, <n_voxels_x>, <n_voxels_y>, <n_voxels_z>) representing a voxelized molecule.
    Notes
    -----
    This function is essentially a wrapper around `moleculekit.tools.voxeldescriptors.getVoxelDescriptors`. 
    Its purpose is to facilitate voxelization of all conformers in a given SmallMol object.
    The default descriptors used to embed each conformer are the same defaults used by `getVoxelDescriptors`.
    User-defined descriptors are supported through the **userchannels** key of the `vargs` parameter i.e. vargs={'userchannels': np.empty((mol.numAtoms, nchannels))}. `vargs` is passed directly to `getVoxelDescriptors`.
    See Also
    --------
    **class** `moleculekit.smallmol.smallmol.SmallMol <https://software.acellera.com/docs/stable/moleculekit/moleculekit.smallmol.smallmol.html?highlight=generateconformers#moleculekit.smallmol.smallmol.SmallMol.generateConformers>`_
    `moleculekit.tools.voxeldescriptors.getVoxelDescriptors <https://software.acellera.com/docs/stable/moleculekit/moleculekit.tools.voxeldescriptors.html?highlight=getvoxeldescriptors#moleculekit.tools.voxeldescriptors.getVoxelDescriptors>`_
    Examples
    --------
    Embed canonical or isomeric SMILES string
        >>> embed_grid('C(C(C(=O)O)N)S')
        >>> embed_grid('C([C@@H](C(=O)O)N)S')
    Embed a .mol2 file by passing a relative path
        >>> embed_grid('benzamidine.mol2')
    Embed a SmallMol object with some defined arguments
        >>> smol = SmallMol('C(C(C(=O)O)N)S')
        >>> cargs = {}
        >>> vargs = {}
        >>> embed_grid(smol, cargs, vargs)
    """
    cargs.setdefault('num_confs', 1)
    vargs.setdefault('userchannels')
    smol = _generateConformers(smol, **cargs)
    confs = [_Conformer(smol, i) for i in range(smol.numFrames)]
    grids = [_embed(conf, **vargs) for conf in confs]
    return np.array(grids)

In [ ]:
def _asArray(x):
    isiter = hasattr(x, '__iter__')
    return np.array([x] if not isiter else x)

def _setdefault(obj, **kwargs):
    for k,v in kwargs.items():
        obj.setdefault(k, v)
    return obj

def voxelize(molecules, embeddings=None, nconformers=1, voxelsize=1, padding=0, preprocess=True, uniform=True, cargs={}, vargs={}):
    """
    Embed a list of small-molecules in a voxelized 3D grid
    Parameters
    ----------
    molecules: list
        A list of SmallMol objects or any objects coercible to SmallMol objects.
    embeddings: list
        A list of functions to used to compute atom properties. The values of each funtion call are concatenated and passed as the `userchannels` argument to `moleculekit.tools.voxeldescriptors.getVoxelDescriptors`.
    nconformers: int
        The number of conforms to generate and embed for each molecule.
    voxelsize: int
        The size of each voxel in angstroms.
    padding: int
        The number of empty voxels to add to each dimension of the grid.
    preprocess: bool
        Apply a preprocessing function to each element in `molecules`.
    uniform: bool
        Embed all molecules in a grid of uniform dimensions defined as the smallest grid that will contain all molecules plus the specified `padding`.
    cargs: dict
        Named arguments passed to `generateConformers`. The value of `num_confs` will overwrite the `nconformers` argument.
    vargs: dict
        Named arguments passed to `getVoxelDescriptors`. This arguments `voxelsize`, `padding`, `boxsize`, and `center` values will be ignored unless the `preprocess` argument is set to False.
    Examples
    --------
    Voxelize a list of SMILES strings
        >>> molecules = ['C([C@@H]1[C@H]([C@@H]([C@H](C(O1)O)O)O)O)O', 'C(C(C(=O)O)N)S', 'CCO']
        >>> voxelize(molecules)
    Voxelize a mixed list of molecular representations
        >>> molecules = ['dextrose.mol2', SmallMol('CCO')]
        >>> voxelize(molecules)
    Pass a custom preprocessing function
        >>> remover = SaltRemover(defnData="[Cl]")
        >>> def no_chloride(smallmol):
        >>>     smallmol._mol = remover.StripMol(smallmol._mol)
        >>>     return smallmol
        >>> voxelize('C[N+]1=CC=CC=C1/C=N/O.[Cl-]', preprocess=no_chloride)
    Pass custom embedding functions and grid properties
        >>> from moleculekit.tools.voxeldescriptors import _getPropertiesRDkit
        >>> molecules = ['C(C(C(=O)O)N)S']
        >>> embeddings = [_getPropertiesRDkit]
        >>> voxelize(molecules, embeddings, 3, 0.5, 10)
    """
    smols = _asArray(molecules)
    smols = [SmallMol(m) for m in smols]
    if preprocess:
        parse = preprocess if callable(preprocess) else _SmallMol
        smols = [parse(m) for m in smols]
    if uniform:
        bbox,bsize,center = min_common_box(smols)
        vargs = _setdefault(vargs, boxsize=bsize, center=center)        
    cargs = _setdefault(cargs, num_confs=nconformers, append=False)
    vargs = _setdefault(vargs, voxelsize=voxelsize, buffer=padding, userchannels=None)
    grids = []
    for m in smols:
        if embeddings:
            vargs['userchannels'] = makeChannels(m, embeddings, unique=True)
        grid = embed_grid(m, cargs, vargs)
        grids.append(grid) 
    return np.array(grids)